# Final Project: Segmentation of satellite images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install --upgrade --no-cache-dir gdown
!gdown 1TVduwykrR1C_VKx2VJcjmwvJtG4HziA1 # set file id
!mkdir /content/data
!unzip data128.zip -d data # set correct file name

## U-Net

In [ ]:
# necessary imports for U-Net:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, Cropping2D, BatchNormalization, Activation, Dropout
from tensorflow.keras import Model, Input
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
import random
import os
from PIL import Image
import tensorflow.keras.backend as K

In [ ]:
def ResBlock(prev_layer, filters, shortcut=False, dropout_rate=0.0):
    if shortcut:
        shortcut_layer = Conv2D(filters, 1, strides = 2, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
        shortcut_layer = BatchNormalization()(shortcut_layer)
        shortcut_layer = Activation("relu")(shortcut_layer)
        prev_layer = Conv2D(filters, 3, strides = 2, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
        prev_layer = BatchNormalization()(prev_layer)
        prev_layer = Activation("relu")(prev_layer)
        prev_layer = Conv2D(filters, 3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
        prev_layer = BatchNormalization()(prev_layer)
        prev_layer = Activation("relu")(prev_layer)
    else:
        shortcut_layer = prev_layer
        prev_layer = Conv2D(filters, 3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
        prev_layer = BatchNormalization()(prev_layer)
        prev_layer = Activation("relu")(prev_layer)
        prev_layer = Conv2D(filters, 3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
        prev_layer = BatchNormalization()(prev_layer)
        prev_layer = Activation("relu")(prev_layer)
    prev_layer = tf.add(prev_layer, shortcut_layer)
    if dropout_rate > 0.0:
        prev_layer = Dropout(dropout_rate)(prev_layer)
    return prev_layer

def EncoderBlock(prev_layer, filters, shortcut=True, dropout_rate=0.0):
    prev_layer = ResBlock(prev_layer, filters, shortcut=shortcut, dropout_rate = dropout_rate)
    prev_layer = ResBlock(prev_layer, filters, dropout_rate = dropout_rate)
    skip_layer = prev_layer
    return prev_layer, skip_layer
    
def DecoderBlock(prev_layer, skip_layer, filters, dropout_rate = 0.0):
    prev_layer = Conv2DTranspose(filters, 2, strides=2, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
    prev_layer = BatchNormalization()(prev_layer)
    prev_layer = Activation("relu")(prev_layer)
    prev_layer = tf.concat([skip_layer, prev_layer], axis=-1)
    prev_layer = Conv2D(filters, 3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
    prev_layer = BatchNormalization()(prev_layer)
    prev_layer = Activation("relu")(prev_layer)
    prev_layer = Conv2D(filters, 3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
    prev_layer = BatchNormalization()(prev_layer)
    prev_layer = Activation("relu")(prev_layer)
    if dropout_rate > 0.0:
        prev_layer = Dropout(dropout_rate)(prev_layer)
    return prev_layer

In [ ]:
def UNet(input_size=(128, 128, 3), n_classes=4, dropout_rate=0.0):
    inputs = Input(shape = input_size)
    conv = Conv2D(64, 7, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(inputs)
    prev_layer = BatchNormalization()(conv)
    prev_layer = Activation("relu")(prev_layer)
    if dropout_rate > 0.0:
        conv = Dropout(dropout_rate)(conv)
    enc1 = EncoderBlock(conv, 64, shortcut=False, dropout_rate=dropout_rate)
    enc2 = EncoderBlock(enc1[0], 128, shortcut=True, dropout_rate=dropout_rate)
    enc3 = EncoderBlock(enc2[0], 256, shortcut=True, dropout_rate=dropout_rate)
    enc4 = EncoderBlock(enc3[0], 512, shortcut=True, dropout_rate=dropout_rate)
    enc5 = EncoderBlock(enc4[0], 1024, shortcut=True, dropout_rate=dropout_rate)
    dec1 = DecoderBlock(enc5[0], enc4[1], 512, dropout_rate=dropout_rate)
    dec2 = DecoderBlock(dec1, enc3[1], 256, dropout_rate=dropout_rate)
    dec3 = DecoderBlock(dec2, enc2[1], 128, dropout_rate=dropout_rate)
    dec4 = DecoderBlock(dec3, enc1[1], 64, dropout_rate=dropout_rate)
    conv = Conv2D(n_classes, 1, activation="softmax", padding="same", kernel_initializer="glorot_normal", bias_initializer="zeros")(dec4)
    model = Model(inputs=inputs, outputs=conv)
    return model

## Dataloader

In [ ]:
# define a dataloader for the training pipe:
class SatelliteData(tf.keras.utils.Sequence):

    FOREST = (4, 135, 29)
    FIELDS = (231, 231, 25)
    URBAN = (229, 109, 109)
    WATER = (14, 10, 214)

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths, shuffle = False):
        self.batch_size = batch_size
        self.img_size = img_size
        if shuffle:
            indices = [*range(len(input_img_paths))]
            random.shuffle(indices)
            self.input_img_paths = np.array(input_img_paths)[indices]
            self.target_img_paths = np.array(target_img_paths)[indices]
        else:
            self.input_img_paths = np.array(input_img_paths)
            self.target_img_paths = np.array(target_img_paths)

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (4,), dtype="float32")
        for j, path in enumerate(batch_target_img_paths):
            img = np.array(load_img(path, target_size=self.img_size))
            y[j] = np.all(img[:, :, None] == (self.FOREST, self.FIELDS, self.URBAN, self.WATER), axis=-1)
        return x, y

In [ ]:
data_path = "data"
drive_path = "drive/MyDrive/final_project"
train_input_dir = os.path.join(data_path, "training", "images")
train_target_dir = os.path.join(data_path, "training", "annotations")
val_input_dir = os.path.join(data_path, "validation", "images")
val_target_dir = os.path.join(data_path, "validation", "annotations")

train_input_img_paths = [os.path.join(train_input_dir, img) for img in os.listdir(train_input_dir)]
train_target_img_paths = [os.path.join(train_target_dir, img) for img in os.listdir(train_target_dir)]

val_input_img_paths = [os.path.join(val_input_dir, img) for img in os.listdir(val_input_dir)]
val_target_img_paths = [os.path.join(val_target_dir, img) for img in os.listdir(val_target_dir)]

print("Number of samples:", len(train_input_img_paths))

In [ ]:
# test dataloader:
def transform_to_rgb(ann):
    FOREST = (4, 135, 29)
    FIELDS = (231, 231, 25)
    URBAN = (229, 109, 109)
    WATER = (14, 10, 214)
    colors = (FOREST, FIELDS, URBAN, WATER)
    rgb_ann = np.zeros(ann.shape[:-1] + (3,), dtype="uint8")
    for i in range(4):
        rgb_ann[ann[:, :, i].astype(bool)] = colors[i]
    return rgb_ann

img_size = (128, 128)
batch_size = 1
train_data = SatelliteData(batch_size, img_size, train_input_img_paths, train_target_img_paths, shuffle = True)
val_data = SatelliteData(batch_size, img_size, val_input_img_paths, val_target_img_paths, shuffle = True)
img, ann = train_data[0]
im = Image.fromarray(np.uint8(img[0]), mode="RGB")
rgb_an = transform_to_rgb(ann[0])
an = Image.fromarray(rgb_an, mode="RGB")
display(im)
display(an)

In [ ]:
# test u-net:
model = UNet()
model.summary()

## Custom loss functions

In [ ]:
class JaccardLoss: #paper?

  def __init__(self, smooth = 1e-5):
    self.smooth = smooth

  def __call__(self, y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=(0, 1, 2))
    y_true = K.sum(y_true, axis=(0, 1, 2))
    y_pred = K.sum(y_pred, axis=(0, 1, 2))
    loss = (intersection + self.smooth) / (y_true + y_pred - intersection + self.smooth) 
    loss = 1.0 - K.mean(loss)
    return loss

class CategoricalFocalLoss: # cite paper

  def __init__(self, gamma = 2.0, alpha = 0.25):
    self.gamma = gamma
    self.alpha = alpha

  def __call__(self, y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
    loss = -1.0 * y_true * self.alpha * K.pow(1.0 - y_pred, self.gamma) * K.log(y_pred)
    loss = K.mean(loss)
    return loss

In [ ]:
class JaccardCoefficient: #paper?

  def __init__(self, name = "jaccard_coefficient", smooth = 1e-5):
    self.__name__ = name
    self.smooth = smooth

  def __call__(self, y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=(0, 1, 2))
    y_true = K.sum(y_true, axis=(0, 1, 2))
    y_pred = K.sum(y_pred, axis=(0, 1, 2))
    jaccard = (intersection + self.smooth) / (y_true + y_pred - intersection + self.smooth) 
    jaccard = K.mean(jaccard)
    return jaccard

## Custom Callbacks

In [ ]:
class BestModelCheckPoint(tf.keras.callbacks.Callback):

    def __init__(self, dirpath):
        self.best_cat_epoch = 0
        self.best_jac_epoch = 0
        self.best_cat = 0.0
        self.best_jac = 0.0
        self.dirpath = dirpath
        self.best_cat_model = None
        self.best_jac_model = None

    def on_epoch_end(self, epoch, logs=None):
        if logs["val_categorical_accuracy"] > self.best_cat:
            self.best_cat = logs["val_categorical_accuracy"]
            self.best_cat_epoch = epoch+1
            self.best_cat_model = self.model.get_weights()
        if logs["val_jaccard_coefficient"] > self.best_jac:
            self.best_jac = logs["val_jaccard_coefficient"]
            self.best_jac_epoch = epoch+1
            self.best_jac_model = self.model.get_weights()

    def on_train_end(self, logs=None):
        cat_model_path = os.path.join(self.dirpath, "cat_model")
        os.mkdir(cat_model_path)
        model_path = os.path.join(self.dirpath, "cat_model", f"{self.best_cat_epoch}_{self.best_cat:.4f}")
        self.model.set_weights(self.best_cat_model)
        self.model.save_weights(model_path)
        jac_model_path = os.path.join(self.dirpath, "jac_model")
        os.mkdir(jac_model_path)
        model_path = os.path.join(jac_model_path, f"{self.best_jac_epoch}_{self.best_jac:.4f}")
        self.model.set_weights(self.best_jac_model)
        self.model.save_weights(model_path)
        print(f"best cat-model saved: {self.best_cat} / {self.best_cat_epoch}, best jac-model saved: {self.best_jac} / {self.best_jac_epoch}")

## Training

In [ ]:
batch_sizes = (32, 64)
dropout_rates = (0.0, 0.1, 0.3)
learning_rates = (0.001, 0.0001, 0.00001)
losses = (CategoricalFocalLoss(), JaccardLoss())
loss_short_names = ("focal", "jaccard")
img_size = (128, 128)

for batch_size in batch_sizes:
    for dropout_rate in dropout_rates:
        for learning_rate in learning_rates:
            for l, loss in enumerate(losses):

                train_data = SatelliteData(batch_size, img_size, train_input_img_paths, train_target_img_paths, shuffle = True)
                val_data = SatelliteData(batch_size, img_size, val_input_img_paths, val_target_img_paths, shuffle = True)

                model_name = "res_model"
                model_name += f"_bs_{batch_size}"
                model_name += f"_lr_{learning_rate}"
                model_name += f"_dr_{dropout_rate}"
                model_name += f"_loss_{loss_short_names[l]}"
                model_path = os.path.join(drive_path, model_name)
                if os.path.exists(model_path):
                    continue
                os.mkdir(model_path)
                os.mkdir(os.path.join(model_path, "logs"))

                check = BestModelCheckPoint(model_path)
                callbacks = [
                    check,
                    tf.keras.callbacks.TensorBoard(log_dir = os.path.join(model_path, "logs"))
                ]
                model = UNet(input_size = img_size + (3,), dropout_rate = dropout_rate)

                model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate), loss = loss, metrics = [tf.keras.metrics.CategoricalAccuracy(), JaccardCoefficient()])
                epochs = 30
                model.fit(train_data, validation_data = val_data, epochs = epochs, callbacks = callbacks)